In [1]:

include("../src/math.jl")
include("../src/experimental.jl")
using MLJ
using MLDataPattern
using DataFrames
using XGBoost
using MLJXGBoostInterface
using Plots
using MultivariateStats
using MLJBase
using Printf
using UMAP
using ThreadSafeDicts
using MLJ
using Printf
using CategoricalArrays
using BioAlignments
using CSV

In [25]:
df = DataFrame(CSV.File("../data/pdb.csv"))

delete!(df, 13)
p = bar(df.Year, [df.pdb,df.seq], label = ["PDB entries" "Number of Uniprot entries added"],  xlabel = "Year", ylabel = "Entries")
savefig(p, "../thesis/Fig/seqs.png")

"/home/mfat/Master-Thesis/thesis/Fig/seqs.png"

In [18]:
import Pkg; Pkg.add("StatPlots")

   Resolving package versions...


Pkg.Resolve.ResolverError: Unsatisfiable requirements detected for package [38;5;11mStatPlots [60ddc479][39m:
 [38;5;11mStatPlots [60ddc479][39m log:
 ├─possible versions are: [38;5;11m0.8.0-0.9.2[39m or uninstalled
 ├─restricted to versions [38;5;11m*[39m by an explicit requirement, leaving only versions: [38;5;11m0.8.0-0.9.2[39m
 └─restricted by julia compatibility requirements to versions: uninstalled — no versions left

In [32]:
using DataFrames
using CSV
using Plots

# Load the data from a CSV file
df = DataFrame(CSV.File("../data/pdb.csv"))
xtick = minimum(df.Year):2:maximum(df.Year)
# Create a plot
p = plot(df.Year, df.pdb, linecolor = :blue, linewidth = 2, xticks=xtick, xlabel = "Year", label = "PDB entries", legend = :topleft, ylabel = "PDB Entries", dpi = 300, title = "Number of PDB and UniParc entries")

# Create a new plot with a shared x-axis and independent y-axis
plot!(twinx(), df.Year, df.seq, linecolor = :red, linewidth = 2, label = "UniParc entries", legend = :bottomright, ylabel = "UniParc Entries")

# Save the combined plot to a file
savefig("../thesis/Fig/combined_plot_lines.png")


"/home/mfat/Master-Thesis/thesis/Fig/combined_plot_lines.png"

In [4]:
using DataFrames
using CSV
using Plots

# Load the data from a CSV file
df = DataFrame(CSV.File("../data/pdb.csv"))

# Create a plot with two subplots
p = plot(layout = (2, 1), size = (800, 600))

# Add the first subplot (PDB entries)
bar!(p[1], df.Year, df.pdb, xlabel = "Year", ylabel = "Entries", yformatter = :plain, xticks = minimum(df.Year):maximum(df.Year), color=:blue, title="Number of PDB entries (a)")

# Add the second subplot (Number of Uniprot entries added)
bar!(p[2], df.Year, df.seq, xlabel = "Year", ylabel = "Entries", yformatter = :plain, xticks = minimum(df.Year):maximum(df.Year), color=:red, title="Number of UniParc entries (b)")

# Save the combined plot to a file
savefig(p, "../thesis/Fig/combined_plots.png")

"/home/mfat/Master-Thesis/thesis/Fig/combined_plots.png"

In [12]:
AA_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
b = [0.0 0.479 0.4995 0.4855 0.498 0.5063 0.4905 0.4929 0.496 0.4935 0.4867 0.4884 0.488 0.4937 0.4977 0.495 0.4859 0.4878 0.4803 0.4855; 0.479 0.0 0.4881 0.4897 0.4972 0.4905 0.4907 0.4877 0.4936 0.4921 0.4881 0.486 0.488 0.4949 0.4941 0.4928 0.4869 0.4888 0.4953 0.4899; 0.4995 0.4881 0.0 0.4884 0.4873 0.4912 0.486 0.4962 0.4929 0.4894 0.4928 0.4919 0.4957 0.4894 0.491 0.4855 0.4964 0.4913 0.4912 0.4902; 0.4855 0.4897 0.4884 0.0 0.4937 0.482 0.4982 0.4824 0.4879 0.4956 0.4924 0.4807 0.4957 0.494 0.4876 0.4981 0.4934 0.4853 0.4976 0.4814; 0.498 0.4972 0.4873 0.4937 0.0 0.4825 0.4965 0.4821 0.4944 0.5023 0.4855 0.4908 0.4788 0.4819 0.4867 0.4928 0.4939 0.4916 0.4895 0.4903; 0.5063 0.4905 0.4912 0.482 0.4825 0.0 0.4946 0.4898 0.4933 0.4888 0.492 0.4911 0.4991 0.4854 0.4914 0.4875 0.4872 0.4893 0.493 0.4814; 0.4905 0.4907 0.486 0.4982 0.4965 0.4946 0.0 0.4914 0.4981 0.4922 0.4948 0.4813 0.4949 0.5032 0.4892 0.4959 0.4976 0.4839 0.4834 0.486; 0.4929 0.4877 0.4962 0.4824 0.4821 0.4898 0.4914 0.0 0.4923 0.4926 0.4914 0.4859 0.4869 0.4904 0.5002 0.4921 0.4946 0.4909 0.4942 0.4878; 0.496 0.4936 0.4929 0.4879 0.4944 0.4933 0.4981 0.4923 0.0 0.4913 0.4903 0.4936 0.496 0.4931 0.4937 0.5014 0.4939 0.4956 0.4971 0.4889; 0.4935 0.4921 0.4894 0.4956 0.5023 0.4888 0.4922 0.4926 0.4913 0.0 0.489 0.5023 0.4885 0.4864 0.4936 0.4937 0.4944 0.4879 0.5002 0.4868; 0.4867 0.4881 0.4928 0.4924 0.4855 0.492 0.4948 0.4914 0.4903 0.489 0.0 0.4897 0.4921 0.4932 0.4942 0.4883 0.4936 0.4923 0.494 0.4984; 0.4884 0.486 0.4919 0.4807 0.4908 0.4911 0.4813 0.4859 0.4936 0.5023 0.4897 0.0 0.4894 0.4881 0.4887 0.4938 0.4893 0.4868 0.4933 0.4817; 0.488 0.488 0.4957 0.4957 0.4788 0.4991 0.4949 0.4869 0.496 0.4885 0.4921 0.4894 0.0 0.5035 0.4873 0.491 0.4875 0.4886 0.4843 0.4893; 0.4937 0.4949 0.4894 0.494 0.4819 0.4854 0.5032 0.4904 0.4931 0.4864 0.4932 0.4881 0.5035 0.0 0.4812 0.4919 0.4904 0.4887 0.492 0.4956; 0.4977 0.4941 0.491 0.4876 0.4867 0.4914 0.4892 0.5002 0.4937 0.4936 0.4942 0.4887 0.4873 0.4812 0.0 0.4985 0.4862 0.4901 0.4974 0.487; 0.495 0.4928 0.4855 0.4981 0.4928 0.4875 0.4959 0.4921 0.5014 0.4937 0.4883 0.4938 0.491 0.4919 0.4985 0.0 0.4853 0.488 0.4915 0.4847; 0.4859 0.4869 0.4964 0.4934 0.4939 0.4872 0.4976 0.4946 0.4939 0.4944 0.4936 0.4893 0.4875 0.4904 0.4862 0.4853 0.0 0.4949 0.4964 0.4904; 0.4878 0.4888 0.4913 0.4853 0.4916 0.4893 0.4839 0.4909 0.4956 0.4879 0.4923 0.4868 0.4886 0.4887 0.4901 0.488 0.4949 0.0 0.4943 0.4881; 0.4803 0.4953 0.4912 0.4976 0.4895 0.493 0.4834 0.4942 0.4971 0.5002 0.494 0.4933 0.4843 0.492 0.4974 0.4915 0.4964 0.4943 0.0 0.4874; 0.4855 0.4899 0.4902 0.4814 0.4903 0.4814 0.486 0.4878 0.4889 0.4868 0.4984 0.4817 0.4893 0.4956 0.487 0.4847 0.4904 0.4881 0.4874 0.0]



heatmap(norm_blosum, title="Target BLOSUM62 distances", xticks = (1:20, AA_list), yticks =  (1:20, AA_list), clims=(0.0,1.0), dpi = 300)
savefig("../thesis/Fig/blosumtarget.png")


"/home/mfat/Master-Thesis/thesis/Fig/blosumtarget.png"

In [2]:
GRANTHAM1974

SubstitutionMatrix{BioSymbols.AminoAcid, Int64}:
             A      R      N      D      C      Q      E      G      H      I      L      K      M      F      P      S      T      W      Y      V      O      U      B      J      Z      X      *
      A      0    112    111    126    195     91    107     60     86     94     96    106     84    113     27     99     58    148    112     64    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲
      R    112      0     86     96    180     43     54    125     29     97    102     26     91     97    103    110     71    101     77     96    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲
      N    111     86      0     23    139     46     42     80     68    149    153     94    142    158     91     46     65    174    143    133    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲    1̲0̲0̲
      D    126     96     23      0    154     61     45     94     81    168    172    101    160  

In [7]:
AA_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

simmat = copy(GRANTHAM1974)
simmatblo = copy(BLOSUM62)

grantham = [[simmat[i, j] for i in AA_list] for j in AA_list]
grantham = hcat(grantham...)

blosum = [[simmatblo[i, j] for i in AA_list] for j in AA_list]
blosum = hcat(blosum...)

function norm_mat(matrix, zero=false)
    if zero == true
        min_value = 0
    else 
        min_value = minimum(matrix)
    end
    
    max_value = maximum(matrix)

    normalized_matrix = (matrix .- min_value) ./ (max_value - min_value)
    return normalized_matrix
end

norm_blosum = broadcast(abs, norm_mat(blosum) .-1)
setindex!.(Ref(norm_blosum), 0.0, 1:20, 1:20)
norm_grant = norm_mat(grantham, true)

20×20 Matrix{Float64}:
 0.0       0.906977  0.586047  0.497674  …  0.297674   0.688372  0.52093
 0.906977  0.0       0.716279  0.790698     0.893023   1.0       0.902326
 0.586047  0.716279  0.0       0.209302     0.706977   0.84186   0.744186
 0.497674  0.790698  0.209302  0.0          0.562791   0.706977  0.567442
 0.525581  0.953488  0.823256  0.651163     0.232558   0.186047  0.102326
 0.27907   0.739535  0.437209  0.455814  …  0.506977   0.855814  0.683721
 0.4       0.809302  0.376744  0.186047     0.390698   0.534884  0.386047
 0.437209  0.92093   0.781395  0.623256     0.134884   0.283721  0.153488
 0.493023  0.939535  0.469767  0.260465     0.451163   0.511628  0.395349
 0.446512  0.92093   0.8       0.64186      0.148837   0.283721  0.167442
 0.390698  0.911628  0.744186  0.586047  …  0.0976744  0.311628  0.167442
 0.516279  0.646512  0.106977  0.195349     0.618605   0.809302  0.665116
 0.125581  0.786047  0.502326  0.432558     0.316279   0.683721  0.511628
 0.423256  0.716

In [15]:
norm_blosum

20×20 Matrix{Float64}:
 0.0       0.733333  0.866667  0.8       …  0.733333  0.933333  0.866667
 0.733333  0.0       0.933333  1.0          0.8       0.866667  0.866667
 0.866667  0.933333  0.0       0.6          0.933333  1.0       0.933333
 0.8       1.0       0.6       0.0          0.866667  0.933333  0.866667
 0.866667  0.866667  0.933333  0.933333     0.8       0.666667  0.533333
 0.733333  0.933333  0.8       0.866667  …  0.933333  0.866667  0.933333
 0.866667  0.933333  0.8       0.733333     0.933333  0.866667  0.6
 0.8       0.8       0.933333  0.933333     0.533333  0.933333  0.8
 0.8       0.933333  0.8       0.666667     0.866667  0.933333  0.866667
 0.8       0.8       1.0       0.933333     0.666667  0.866667  0.8
 0.8       0.8       0.933333  0.866667  …  0.666667  0.8       0.8
 0.866667  0.933333  0.666667  0.733333     0.933333  1.0       0.866667
 0.8       0.933333  0.8       0.8          0.866667  1.0       0.933333
 0.8       0.933333  0.733333  0.6          0.86

In [2]:
using JLD
using Evolutionary
using Distances
using Random
using BioAlignments
using TickTock

function fitness_func(individual, distance_matrix, dim)
    num_vectors = size(distance_matrix, 1)
    binary_vectors = transpose(reshape(individual, (dim, num_vectors)))
    distances = zeros((num_vectors, num_vectors))
    
    Threads.@threads for i in 1:num_vectors
        for j in 1:num_vectors
            if i != j
                distances[i, j] = Distances.hamming(Vector{Int}(binary_vectors[i, :]), Vector{Int}(binary_vectors[j, :]))
            end
        end
    end
    
    return sum((distances - distance_matrix).^2)
end

function generate_initial_population(population_size, dim, num_vectors)
    return [bitrand(dim*num_vectors) for i in 1:population_size]
end

function generate_binary_vectors(dim, num_vectors, distance_matrix, max_generations=300, population_size=100, mutation_rate = 0.05)

    opts = Evolutionary.Options(iterations=max_generations, parallelization=:thread, show_trace=true)

    ga = GA(
        populationSize=population_size,
        crossoverRate=0.8,
        mutationRate=mutation_rate,
        crossover=SPX,
        mutation=flip
    )
    
    objective = individual -> fitness_func(individual, distance_matrix, dim)

    initial_population = generate_initial_population(population_size, dim, num_vectors)

    result = Evolutionary.optimize(objective, initial_population, ga, opts)
    best_individual = result.minimizer
    best_solution = reshape(best_individual, (dim, num_vectors))

    return transpose(best_solution)
end

using Printf

function similarity_error(desired_distance_matrix, calculated_distance_matrix)
    return sum((calculated_distance_matrix .- desired_distance_matrix).^2)
end

function find_best_binary_vectors(dim, num_vectors, distance_matrix, num_trials, max_generations=300, population_size=100, mutation_rate = 0.15)
    best_error = Inf
    best_vectors = nothing
    best_calculated_distance_matrix = nothing

    for trial in 1:num_trials
        binary_vectors = generate_binary_vectors(dim, num_vectors, distance_matrix, max_generations, population_size, mutation_rate)
        calculated_distance_matrix = zeros((num_vectors, num_vectors))

        for i in 1:num_vectors
            for j in 1:num_vectors
                if i != j
                    hamming_distance = hamming(Vector{Int}(binary_vectors[i, :]), Vector{Int}(binary_vectors[j, :]))
                    calculated_distance_matrix[i, j] = hamming_distance / dim
                end
            end
        end

        error = similarity_error(distance_matrix, calculated_distance_matrix)

        if error < best_error
            best_error = error
            best_vectors = binary_vectors
            best_calculated_distance_matrix = calculated_distance_matrix
        end
    end

    return best_vectors, best_calculated_distance_matrix, best_error
end

AA_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

simmat = copy(GRANTHAM1974)
simmatblo = copy(BLOSUM62)

grantham = [[simmat[i, j] for i in AA_list] for j in AA_list]
grantham = hcat(grantham...)

blosum = [[simmatblo[i, j] for i in AA_list] for j in AA_list]
blosum = hcat(blosum...)

function norm_mat(matrix, zero=false)
    if zero == true
        min_value = 0
    else 
        min_value = minimum(matrix)
    end
    
    max_value = maximum(matrix)

    normalized_matrix = (matrix .- min_value) ./ (max_value - min_value)
    return normalized_matrix
end

norm_blosum = broadcast(abs, norm_mat(blosum) .-1)
setindex!.(Ref(norm_blosum), 0.0, 1:20, 1:20)
norm_grant = norm_mat(grantham, true)

# Generate binary vectors using the desired pairwise similarities
distance_matrix = norm_blosum
dim = 100
num_vectors = 20
num_trials = 1
population_size = 2000
max_generations = 200
tick()
best_vectors, best_calculated_distance_matrix, best_error = find_best_binary_vectors(dim, num_vectors, distance_matrix, num_trials, max_generations, population_size)

println("Desired pairwise similarities:")
println(norm_blosum)

println("\nBest calculated pairwise similarities:")
println(best_calculated_distance_matrix)

@printf("\nBest error: %f", best_error)

┌ Info:  started timer at: 2023-05-23T15:15:37.953
└ @ TickTock /home/mfat/.julia/packages/TickTock/fGILW/src/TickTock.jl:54


Iter     Function value
     0   873648.7288888888
 * time: 5.602836608886719e-5


     1   871801.3955555554
 * time: 0.8819601535797119


     2   872779.2622222223
 * time: 1.688621997833252


     3   868736.462222222
 * time: 2.465327024459839


...

   197   386417.79555555555
 * time: 172.79771208763123


   198   382893.66222222225
 * time: 173.63552808761597


   199   382403.5288888889
 * time: 174.52487802505493


   200   382311.928888889
 * time: 175.44456911087036


Desired pairwise similarities:
[0.0 0.7333333333333334 0.8666666666666667 0.8 0.8666666666666667 0.7333333333333334 0.8666666666666667 0.8 0.8 0.8 0.8 0.8666666666666667 0.8 0.8 0.8 0.6666666666666667 0.7333333333333334 0.7333333333333334 0.9333333333333333 0.8666666666666667; 0.7333333333333334 0.0 0.9333333333333333 1.0 0.8666666666666667 0.9333333333333333 0.9333333333333333 0.8 0.9333333333333333 0.8 0.8 0.9333333333333333 0.9333333333333333 0.9333333333333333 0.9333333333333333 0.8 0.8 0.8 0.8666666666666667 0.8666666666666667; 0.8666666666666667 0.9333333333333333 0.0 0.6 0.9333333333333333 0.8 0.8 0.9333333333333333 0.8 1.0 0.9333333333333333 0.6666666666666667 0.8 0.7333333333333334 0.8666666666666667 0.7333333333333334 0.8 0.9333333333333333 1.0 0.9333333333333333; 0.8 1.0 0.6 0.0 0.9333333333333333 0.8666666666666667 0.7333333333333334 0.9333333333333333 0.6666666666666667 0.9333333333333333 0.8666666666666667 0.7333333333333334 0.8 0.6 0.7333333333333334 0.7333333333333334 0